In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

In [6]:
path_info(main_path).sort_values("time")

,path,name,time,size
25,/home/dima/.profile,.profile,2021-01-25 09:33:29.358608246,0.000807
21,/home/dima/.bash_logout,.bash_logout,2021-01-25 09:33:29.358608246,0.000220
56,/home/dima/.logs,.logs,2021-01-25 09:39:17.514788151,0.004096
26,/home/dima/remount_data.sh,remount_data.sh,2021-01-25 09:39:25.638919115,0.000146
22,/home/dima/.sudo_as_admin_successful,.sudo_as_admin_successful,2021-01-25 09:48:01.709587097,0.000000
...,...,...,...,...
12,/home/dima/.zsh_history,.zsh_history,2022-03-26 20:25:58.725558519,0.006565
50,/home/dima/re_foto,re_foto,2022-03-26 21:36:30.745478630,0.004096
35,/home/dima/food,food,2022-03-26 21:43:26.146949530,0.004096
5,/home/dima/.bash_history,.bash_history,2022-03-26 21:54:23.297301054,0.048750


In [ ]:
pd.read_json('')